In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import functools
import pickle
import os
import csv
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn import model_selection
from tqdm.notebook import tqdm

BASE_DIR = '../../../'
import sys
sys.path.append(BASE_DIR)

# custom code
import utils.utils
CONFIG = utils.utils.load_config("../../config.json")
from utils.fweg import FWEG

Using TensorFlow backend.


In [3]:
DATASET = os.path.basename(os.getcwd()) # name of folder this file is in
RANDOM_SEED = CONFIG['random_seed']
BATCH_SIZE = CONFIG["experiment_configs"][DATASET]["batch_size"]

print(RANDOM_SEED)

PROCESSED_DIR = os.path.join(BASE_DIR, f'processed/adult/rs={RANDOM_SEED}')
MODELS_DIR = os.path.join(BASE_DIR, f'models/adult/rs={RANDOM_SEED}')
RESULTS_DIR = os.path.join(BASE_DIR, "results")

PROCESSED_SAVEPATH = utils.utils.get_savepath(PROCESSED_DIR, "adult", ".pkl")
BASE_MODEL_SAVEPATH = utils.utils.get_savepath(MODELS_DIR, "adult", ".h5", mt="base") # mt = model_type


35


In [4]:
os.makedirs(RESULTS_DIR, exist_ok=True)

In [5]:
train_df = pd.read_csv(os.path.join(PROCESSED_DIR, "train.csv"))
hyper_train_df = pd.read_csv(os.path.join(PROCESSED_DIR, "hyper_train.csv"))
val_df = pd.read_csv(os.path.join(PROCESSED_DIR, "val.csv"))
# in this setting val = hyper_val. the separation is purely for consistency with
# other experiments and using FWEG
hyper_val_df = pd.read_csv(os.path.join(PROCESSED_DIR, "hyper_val.csv"))
test_df = pd.read_csv(os.path.join(PROCESSED_DIR, "test.csv"))

# concat the two for train
train_full_df = pd.concat([train_df, hyper_train_df])

del train_df, hyper_train_df

In [6]:
x_train_full = train_full_df.drop(['label'], axis=1).values
y_train_full = train_full_df['label'].values

x_val = val_df.drop(['label'], axis=1).values
y_val = val_df['label'].values

x_hyper_val = hyper_val_df.drop(['label'], axis=1).values
y_hyper_val = hyper_val_df['label'].values

x_test = test_df.drop(['label'], axis=1).values
y_test = test_df['label'].values

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=x_train_full.shape[1]),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax),
])
model.load_weights(BASE_MODEL_SAVEPATH)

In [8]:
preds_train_full = model.predict(x_train_full)
preds_val = model.predict(x_val)
preds_hyper_val = model.predict(x_hyper_val)
preds_test = model.predict(x_test)

In [9]:
def get_basis_fns(
    groups,
    train_df,
    val_df,
    hyper_val_df,
    test_df,
    add_all
    ):
    """
    Get the basis functions for adult.
    """
    all_groups = [
        [],
        ['private_workforce', 'non_private_workforce'],
        ['private_workforce', 'non_private_workforce', 'income']
    ]
    assert groups in all_groups
    if len(groups) == 0:
        assert add_all is True
        
    np.random.seed(RANDOM_SEED)
    
    if len(groups) == 0:
        basis_train = pd.DataFrame(np.ones(len(train_df)), columns=["All"])
        basis_val = pd.DataFrame(np.ones(len(val_df)), columns=["All"])
        basis_hyper_val = pd.DataFrame(np.ones(len(hyper_val_df)), columns=["All"])
        basis_test = pd.DataFrame(np.ones(len(test_df)), columns=["All"])
    else:
        basis_train = train_df[groups].copy()
        basis_val = val_df[groups].copy()
        basis_hyper_val = hyper_val_df[groups].copy()
        basis_test = test_df[groups].copy()
    
        if add_all:
            basis_train['All'] = 1.0
            basis_val['All'] = 1.0
            basis_hyper_val['All'] = 1.0
            basis_test['All'] = 1.0
        
    return basis_train, basis_val, basis_hyper_val, basis_test


In [10]:
CLASSES = 2
METRIC = 'G-mean'

# must be one of:
# []
# ['private_workforce', 'non_private_workforce']
# ['private_workforce', 'non_private_workforce', 'income']
GROUPS = ["private_workforce", "non_private_workforce"]

ADD_ALL = False
EPSILON = 1e-4

NUM_ITERS = 100

USE_LINEAR_VAL_METRIC = True

In [11]:
basis_train_full, basis_val, basis_hyper_val, basis_test = get_basis_fns(
    GROUPS,
    train_full_df,
    val_df,
    hyper_val_df,
    test_df,
    ADD_ALL
)

In [12]:
fweg = utils.fweg.FWEG(
    METRIC,
    NUM_ITERS,
    EPSILON,
    CLASSES,
    USE_LINEAR_VAL_METRIC,
    RANDOM_SEED,
)

val_train_list, grad_norm_list, cond_list = fweg.fit(
    preds_train_full,
    y_train_full,
    basis_train_full,
    preds_val,
    y_val,
    basis_val,
)

Initialization complete!


Val G-mean: 0.685: 100%|██████████| 100/100 [00:01<00:00, 51.58it/s]


In [13]:
# apply to hyper val set
preds_hyper_val_list, mval_hyper_val_list = fweg.predict(
    preds_hyper_val,
    y_hyper_val,
    basis_hyper_val,
    deterministic=False,
)

start = len(mval_hyper_val_list)//2
best_idx = start + np.argmax(mval_hyper_val_list[start:])
print(f"Hyper Val: {mval_hyper_val_list[best_idx]}")

Hyper Val: 0.68476343839989


In [14]:
# apply to test set
preds_test_list, mval_test_list = fweg.predict(
    preds_test,
    y_test,
    basis_test,
    deterministic=False,
)
print(f"Test: {mval_test_list[best_idx]}")

Test: 0.6850109038751132


Run below to try many hyperparams.

In [15]:
os.makedirs(RESULTS_DIR, exist_ok=True)
savepath = os.path.join(RESULTS_DIR, f"results_{DATASET}.csv")
saver = utils.record.Results_Recorder(savepath, DATASET)

Results file exists, appending to it...


In [16]:
CLASSES = 2
NUM_ITERS = 100
METRIC = "G-mean"

groups_list = [
    [],
    ['private_workforce', 'non_private_workforce'],
    ['private_workforce', 'non_private_workforce', 'income']
]
groups_descr_list = [
    "single_group",
    "workforce, non-private workforce",
    "workforce, non-private workforce, income",
]
add_all_list = [False, True]
epsilon_list = [0.0001, 0.001, 0.1, 1.0]
use_linear_val_metric_list = [False, True]

# this fills in most of the arguments for our basis function creator
# it is missing the `groups` arg and `add_all`. FWEG_Hyperparameter_Search
# is given basis_fn_generator and will fill these in as it iterates over
# the hyperparameters.
basis_fn_generator = functools.partial(
    get_basis_fns,
    train_df = train_full_df,
    val_df = val_df,
    hyper_val_df = hyper_val_df,
    test_df = test_df,
)

In [17]:
fweg_hp_s = utils.fweg.FWEG_Hyperparameter_Search(
    saver,
    CLASSES,
    NUM_ITERS,
    METRIC,
    basis_fn_generator,
    groups_list,
    groups_descr_list,
    add_all_list,
    epsilon_list,
    use_linear_val_metric_list,
    RANDOM_SEED,
    use_convergence=True
)

(best_groups, best_add_all, best_epsilon, best_use_linear_val_metric) = fweg_hp_s.search(
    preds_train_full,
    y_train_full,
    preds_val,
    y_val,
    preds_hyper_val,
    y_hyper_val,
    preds_test,
    y_test,
)

best hyper val: 0.721, test: 0.6791:  50%|█████     | 24/48 [00:22<00:43,  1.80s/it] 

UNEXPECTED - already had a trivial classifier


best hyper val: 0.721, test: 0.6791:  52%|█████▏    | 25/48 [00:25<00:48,  2.12s/it]

UNEXPECTED - already had a trivial classifier


best hyper val: 0.721, test: 0.6791:  56%|█████▋    | 27/48 [00:30<00:52,  2.51s/it]

...

best hyper val: 0.721, test: 0.6791:  96%|█████████▌| 46/48 [01:34<00:08,  4.06s/it]

UNEXPECTED - already had a trivial classifier


best hyper val: 0.721, test: 0.6791:  98%|█████████▊| 47/48 [01:38<00:04,  4.12s/it]

UNEXPECTED - already had a trivial classifier


best hyper val: 0.721, test: 0.6791: 100%|██████████| 48/48 [01:42<00:00,  2.14s/it]


In [18]:
saver.close()